In [1]:
from os.path import dirname, join as pjoin
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from brainflow.data_filter import DataFilter, AggOperations, WaveletTypes, NoiseEstimationLevelTypes, WaveletExtensionTypes, ThresholdTypes, WaveletDenoisingTypes
from brainflow.data_filter import DataFilter, FilterTypes, AggOperations, NoiseTypes, DetrendOperations, WindowOperations
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds

In [2]:
mat_contents = sio.loadmat('Data/U001x.mat')
labels_adaptation = [4, 2, 1, 0, 3, 4, 1, 0, 3, 2]
eeg_electrodes = [i.item() for i in mat_contents['info']['label'].item()[0]]

In [3]:
eeg_data = np.array(mat_contents['eeg'])
eeg_data.shape

(15, 16640)

In [4]:
events = mat_contents['events'][:,1]
start_times = np.where(events==32779)[0]
end_times = np.where(events==32780)[0]
print("Finding the index of start and end times")
print(start_times)
print(end_times)

print()
print("Getting the EEG time index of start and end times")
event_index = mat_contents['events'][:,2]
start_index = event_index[start_times]
end_index = event_index[end_times]
print(start_index)
print(end_index)

Finding the index of start and end times
[ 2  5  8 11 14 17 20 23 26 29]
[ 3  6  9 12 15 18 21 24 27 30]

Getting the EEG time index of start and end times
[ 2699  4107  5515  6923  8331  9739 11147 12555 13963 15371]
[ 3339  4747  6155  7563  8971 10379 11787 13195 14603 16011]


In [5]:
eeg_epoch = []
processed_eeg = np.array([eeg_data[i].flatten() for i in range(14)])

# Average re-referencing
means = np.mean(processed_eeg, axis=0)
processed_eeg = processed_eeg - means


for i in range(14):
    DataFilter.perform_bandpass(processed_eeg[i], 128, 1.0, 48.0, 5, FilterTypes.BUTTERWORTH_ZERO_PHASE, 0)
    # DataFilter.perform_bandstop(eeg_data[i], 128, 58.0, 62.0, 2, FilterTypes.BUTTERWORTH_ZERO_PHASE, 0)

print(len(processed_eeg[13]))

for i in range(len(start_index)):
    # Indexing 0:14 to remove last column (it is not eeg signal, just a status signal)
    eeg_epoch.append(processed_eeg[:, start_index[i]:end_index[i]])

eeg_epoch = np.array(eeg_epoch)
print(eeg_epoch.shape)

16640
(10, 14, 640)


In [6]:
def process_and_epoch(mat_file):
    
    # Get EEG Channels
    eeg_data = np.array(mat_file['eeg'])
    
    # Get the index of when the starting and ending events occur
    events = mat_file['events'][:,1]
    start_times = np.where(events==32779)[0]
    end_times = np.where(events==32780)[0]


    # Get the index of when the start and end of trial are in the EEG dataset
    event_index = mat_file['events'][:,2]
    start_index = event_index[start_times]
    end_index = event_index[end_times]

    # Process EEG data set
    eeg_epoch = []
    processed_eeg = np.array([eeg_data[i].flatten() for i in range(14)])

    # Average re-referencing
    means = np.mean(processed_eeg, axis=0)
    processed_eeg = processed_eeg - means

    # Band-pass
    for i in range(14):
        DataFilter.perform_bandpass(processed_eeg[i], 128, 1.0, 48.0, 5, FilterTypes.BUTTERWORTH_ZERO_PHASE, 0)
        # DataFilter.perform_bandstop(eeg_data[i], 128, 58.0, 62.0, 2, FilterTypes.BUTTERWORTH_ZERO_PHASE, 0)

    # Epoch data
    for i in range(len(start_index)):
        # Indexing 0:14 to remove last column (it is not eeg signal, just a status signal)
        eeg_epoch.append(processed_eeg[:, start_index[i]:end_index[i]])

    eeg_epoch = np.array(eeg_epoch)
    print(eeg_epoch.shape)
    
    return eeg_epoch

    

In [7]:
eeg = process_and_epoch(mat_contents)

(10, 14, 640)


In [8]:
import numpy as np
import os

# Example data (3 epochs, 2 channels, 5 time points)
epochs = np.random.rand(3, 2, 5)

# Create a directory to save the data
output_dir = "eeg_epochs/"
os.makedirs(output_dir, exist_ok=True)

# Save each epoch as a separate .npy file
for i, epoch in enumerate(epochs):
    file_path = os.path.join(output_dir, f"epoch_{i}.npy")
    np.save(file_path, epoch)
    print(f"Saved {file_path}")

Saved eeg_epochs/epoch_0.npy
Saved eeg_epochs/epoch_1.npy
Saved eeg_epochs/epoch_2.npy


In [19]:
x = []
x.append(np.array([(1,2,3)]))
y = np.array([(1, 2, 3), (4, 5, 6)])
x = np.concatenate((x,y))
x = np.concatenate((x,y))
x

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 3 dimension(s) and the array at index 1 has 2 dimension(s)